In [1]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileRaw,FileRaw_mapped,SimradDatagram_type_from_string,datagram_type_to_string,t_SimradDatagramType
import themachinethatgoesping.echosounders.simrad as simrad
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import themachinethatgoesping.tools as tools
import themachinethatgoesping.navigation.nmea_0183 as NMEA
import numpy
import os, sys
from time import time
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np

#load files
folders =  ["/home/data/"]
#folders =  ["/home/data/turbeams/"]
folders = ["/home/data/no_backup/ek80_test/"]

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [2]:
fm = FileRaw_mapped(files, show_progress=True)
fm

indexing files ⠐ 100% :00s<00m:00s] [Found: 12382 packages in 26 files (2617MB)]                                            


FileSimradRaw
#############
- Number of loaded .raw files: : 26      
- Total file size: :             2.56 GB 

 Detected datagrams 
-------------------- 
- Total:           12382 
- Packages [NME0]: 6511  [NMEA text datagram]
- Packages [XML0]: 2338  [XML0 text datagram]
- Packages [MRU0]: 1143  [Motion binary datagram]
- Packages [FIL1]: 104   [Filter binary datagram]
- Packages [RAW3]: 2286  [Sample binary datagram]

In [3]:
#%%timeit -n1 -r1
A = defaultdict(list)
for i,nm in enumerate(tqdm(fm("XML0",0,-1))):    
    A[nm.get_xml_datagram_type()].append( (nm,i))

  0%|          | 0/2338 [00:00<?, ?it/s]

In [4]:
for k,v in A.items():
    print(f"{k}: {len(v)}")

Configuration: 26
Environment: 26
Parameter: 2286


In [7]:
times =[("start",time())]
# simrad.test_speed_raw(fm,"XML0")
# times.append(("raw     ",time()))
# simrad.test_speed_type(fm,"XML0")
# times.append(("type    ",time()))
simrad.test_speed_decode_xml(fm,0)
times.append(("decode 0",time()))
simrad.test_speed_decode_xml(fm,1)
times.append(("decode 1",time()))
simrad.test_speed_decode_xml(fm,2)
times.append(("decode 2",time()))
simrad.test_speed_decode_xml(fm,3)
times.append(("decode 3",time()))
simrad.test_speed_decode_xml(fm,4)
times.append(("decode 4",time()))
simrad.test_speed_decode_xml(fm,5)
times.append(("decode 5",time()))
print()
for i in range(1,len(times)):
    dt = (times[i][1] - times[i-1][1])*1000
    it = len(fm("XML0")) / dt
    
    print(F"case {times[i][0]}: {round(dt,2)} ms | {round(it,2)} it/ms")

test reading ⠠ 100% :00s<00m:00s] [time: 7.495402ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 6.167178ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 6.958286ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 8.150736ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 16.558849ms]                   
test reading ⠠ 100% :00s<00m:00s] [time: 8.717576ms]                  

case decode 0: 8.68 ms | 269.42 it/ms
case decode 1: 7.25 ms | 322.68 it/ms
case decode 2: 7.98 ms | 292.95 it/ms
case decode 3: 9.11 ms | 256.76 it/ms
case decode 4: 17.72 ms | 131.94 it/ms
case decode 5: 9.97 ms | 234.48 it/ms


In [ ]:
xml = A["Configuration"][-1][0].decode().Transceivers[0].Channels[0]
xml

In [ ]:
xml = A["Configuration"][0][0].raw().children("ConfiguredSensors")
xml

In [ ]:
xml = A["Sensor"][10][0].decode()
xml

In [ ]:
xml = A["Environment"][10][0].decode().Transducer
xml

In [7]:
xml = A["Configuration"][-1][0].decode()